# Fun con decoradores

Un decorador básico

In [1]:
def midecorador(function):
    def wrapper():
        print ("INICIO")
        function()
        print ("FIN")
    return wrapper

In [2]:
def hola_mundo():
    print (f"Hola Mundo")

In [3]:
midecorador(hola_mundo)()

INICIO
Hola Mundo
FIN


Esto no es la forma de hacer esto en python. Hay que hacer lo siguiente

In [4]:
def midecorador(function):
    def wrapper():
        print ("INICIO")
        function()
        print ("FIN")
    return wrapper

@midecorador
def hola_mundo():
    print (f"Hola Mundo")
    
hola_mundo()

INICIO
Hola Mundo
FIN


y si función tiene argumentos esto va a fallar

In [5]:
@midecorador
def hola_tu(nombre):
    print (f"Hola {nombre}")

hola_tu("Pedro")

TypeError: midecorador.<locals>.wrapper() takes 0 positional arguments but 1 was given

Esto pasa porque el wrapper no acepta las variables, entonces le agregamos los args y kwargs

In [ ]:
def midecorador(function):
    def wrapper(*args, **kwargs):
        print ("INICIO")
        function(*args, **kwargs)
        print ("FIN")
    return wrapper

@midecorador
def hola_tu(nombre):
    print (f"Hola {nombre}")

hola_tu("Pedro")
    

INICIO
Hola Pedro
FIN


Otro ejemplo con logging

In [ ]:
def logged(function):
    def wrapper (*args, **kwargs):
        value = function(*args, **kwargs)
        with open('logfile.txt', 'a+') as f:
            fname = function.__name__ #nombre de la funcion
            f.write(f"LOG: {fname} devuelve valor {value}\n")
            print(f"LOG: {fname} devuelve valor {value}")
        return value
    return wrapper
    
@logged
def add(x,y):
    return x+y

print(add(10,20))
print (add(4,3))

LOG: add devuelve valor 30
30
LOG: add devuelve valor 7
7


Otro ejemplo con timing

In [ ]:
import time

def timed(function):
    def wrapper(*args, **kwargs):
        before = time.perf_counter()
        value = function(*args, **kwargs)
        after = time.perf_counter()
        function_name = function.__name__
        print (f"{function_name} demoró {after-before} segundos en ejecutar")
        return value
    return wrapper

@timed
def myfuncionlenta(x):
    result = 1
    for i in range(1,x):
        result *=i
    return result

print (myfuncionlenta(100))    
print (myfuncionlenta(1_000))    
# no resulta para un númerpo muy grande por un límite de tamaño que no entiendo bien, pero d lo mismo

myfuncionlenta demoró 8.917000059227576e-06 segundos en ejecutar
933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000
myfuncionlenta demoró 0.00040704200000618584 segundos en ejecutar
4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808

### functools.wraps
Otro ejemplo

In [6]:
from functools import wraps
from time import perf_counter, sleep

def do_something(param: str):
    ''' Esto es una función importante'''
    sleep(1)
    print (param)
    
do_something("Hola")

Hola


Nada nuevo acá, pero lo que queremos es tomarle el tiempo

In [10]:
from functools import wraps
from time import perf_counter, sleep

def get_time(func):
    '''toma tiempo de funcion'''
    
    def wrapper (*args, **kwargs):
        start_time = perf_counter()
        
        print (*args, **kwargs)
        func(*args, **kwargs)
        
        end_time = perf_counter()
        
        total_time = round(end_time - start_time, 2)
        
        print (f"Total time {total_time} segundos")
    
    return wrapper
        
@get_time
def do_something(param: str):
    ''' Esto es una función importante'''
    sleep(1)
    print (param)
    
do_something("Hola")
print ("="*10)

@get_time
def do_something(param: str, lst: list):
    ''' Esto es una función importante'''
    sleep(1)
    print (param)
do_something("Hello", ["Hello",3])

Hola
Hola
Total time 1.0 segundos
Hello ['Hello', 3]
Hello
Total time 1.01 segundos


In [11]:
@get_time
def do_something(param: str):
    ''' Esto es una función importante'''
    sleep(1)
    print (param)
    
print (do_something.__name__)

wrapper


Transfiere el nombre al wrapper y esto no está bien

In [12]:
print (do_something.__doc__)

None


In [17]:
from functools import wraps
from time import perf_counter, sleep

def get_time(func):
    '''toma tiempo de funcion'''
    
    @wraps(func) #transfiere el dato desde func
    def wrapper (*args, **kwargs):
        start_time = perf_counter()
        
        print (*args, **kwargs)
        func(*args, **kwargs)
        
        end_time = perf_counter()
        
        total_time = round(end_time - start_time, 2)
        
        print (f"Total time {total_time} segundos")
    
    return wrapper
        
@get_time
def do_something(param: str):
    '''Esto es una función importante'''
    sleep(1)
    print (param)

do_something("HOLANDA")
print (do_something.__name__)
print (do_something.__doc__)
    

HOLANDA
HOLANDA
Total time 1.0 segundos
do_something
Esto es una función importante
